## Thread Pool Executor

#### 1. max_workers

- The number of threads in your pool
- More workers = more tasks running simultaneously
- Too many workers can slow things down (overhead)
- Good rule: `max_workers = 5 to 10` for I/O tasks

#### 2. submit() vs map()

**Use `submit()` when:**
- Tasks are different
- You need fine control
- Tasks have different arguments
```python
executor.submit(download, "url1")
executor.submit(process, data1, data2)
executor.submit(calculate, 100)

Use `map()` when:

Same function, different data
Processing a list/collection
Want results in order

pythonexecutor.map(download, url_list)
3. Future Objects
When you call submit(), you get a Future:
pythonfuture = executor.submit(some_function)
result = future.result()  # Blocks until done
Think of a Future as a "promise" - it promises to give you a result later.
4. When to Use ThreadPoolExecutor?
✅ GOOD FOR (I/O-bound tasks):

Downloading files/web pages
Reading/writing files
Database queries
API calls
Network operations

In [ ]:
from concurrent.futures import ThreadPoolExecutor                         
import time 
import random

coins = ["bitcoin", "dogecoin", "sushi swap"]

def task(name):
    print(f"Starting task {name}")
    time.sleep(2)
    print(f"Finished task {name}")
    return f"{name} got: {random.choice(coins)}"

with ThreadPoolExecutor(max_workers=3) as executor:
    future1 = executor.submit(task, "A")
    future2 = executor.submit(task, "B")
    future3 = executor.submit(task, "C")
    future4 = executor.submit(task, "d")
    future5 = executor.submit(task, "e")
    future6 = executor.submit(task, "f")
    print(future1.result())
    print(future2.result())
    print(future3.result())
    print(future4.result())
    print(future5.result())
    print(future6.result())                                              
    
    

Starting task A
Finished task A
Starting task B
A got: sushi swap
Finished task B
Starting task C
B got: dogecoin
Finished task C
Starting task d
C got: bitcoin
Finished task d
Starting task e
d got: dogecoin
Finished task e
Starting task f
e got: sushi swap
Finished task f
f got: bitcoin


In [23]:
from concurrent.futures import ThreadPoolExecutor
import time

def square_of(n):
    print(f"Computing Square of {n}")
    time.sleep(1)
    return n * n
    
numbers = [2, 4, 8, 5, 24, 97, 3]

with ThreadPoolExecutor(max_workers=2) as executor:
    result = executor.map(square_of, numbers)
    for res in result:
        print(f"\nSquare of {int(res ** 0.5)} is: {res}")
        

Computing Square of 2
Computing Square of 4
Computing Square of 8
Square of 2 is: 4

Computing Square of 5

Square of 4 is: 16
Computing Square of 24
Square of 8 is: 64

Computing Square of 97

Square of 5 is: 25
Computing Square of 3

Square of 24 is: 576

Square of 97 is: 9409

Square of 3 is: 9


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

def slow_add(x, y):
    time.sleep(3)
    return x + y

with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(slow_add, 5, 20)
    print("Task Submitted: Checking Status\n")
    
    print(f"Is done: {future.done()}")
    
    result = future.result()
    
    print(f"Is done: {future.done()}")
    print(f"Result: {result}")

Task Submitted: Checking Status

Is done: False
Is done: True
Result: 25


In [27]:
from concurrent.futures import ThreadPoolExecutor
import time

def divide(x, y):
    time.sleep(2)
    return x / y

with ThreadPoolExecutor(max_workers=1) as executor:
    f1 = executor.submit(divide, 10, 5)
    f2 = executor.submit(divide, 10, 0)
    
    print(f"F1 result: {f1.result()}")
    try:
        print(f"F2 result: {f2.result()}")
    except Exception as e:
        print(f"F2 exception caught: {e}")
        
    print(f"F2 exception: {f2.exception()}")

F1 result: 2.0
F2 exception caught: division by zero
F2 exception: division by zero


In [29]:
from concurrent.futures import ThreadPoolExecutor
import time

def sleepy_task(n):
    time.sleep(n)
    return f"Slept {n} seconds"

with ThreadPoolExecutor(max_workers=1) as executor:
    future1 = executor.submit(sleepy_task, 5) 
    future2 = executor.submit(sleepy_task, 3)  

    cancelled = future2.cancel()
    print("Cancelled future2?", cancelled)

    print("future1 result:", future1.result())  
    print("future2 cancelled?", future2.cancelled())


Cancelled future2? True
future1 result: Slept 5 seconds
future2 cancelled? True


In [31]:
from concurrent.futures import ThreadPoolExecutor
import time

def work(x):
    time.sleep(2)
    return x * 2

def notify_done(fut):
    print("Callback: task finished with result:", fut.result())

with ThreadPoolExecutor(max_workers=2) as executor:
    future = executor.submit(work, 5)
    future.add_done_callback(notify_done)

    print("Task submitted... waiting for result")
    result = future.result()
    print("Final result:", result)



Task submitted... waiting for result
Callback: task finished with result: 10
Final result: 10


In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait, FIRST_COMPLETED
import time

def task(n):
    time.sleep(n)
    return f"Task {n} done"

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(task, i) for i in [5, 3, 1]]
    
    done, not_done = wait(futures, return_when=FIRST_COMPLETED)
    print(f"[First Finished]: {[f.result() for f in done]}")
    
    done2, not_done2 = wait(not_done)
    print(f"All finished: {[f.result() for f in done2]}")
    

[First Finished]: ['Task 3 done']
All finished: ['Task 5 done', 'Task 3.01 done']


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time, random

def work(n):
    delay = random.randint(1, 4) 
    print(f"Task {n} sleeping {delay}s")
    time.sleep(delay)
    return f"Task {n} finished in {delay}s"

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(work, i) for i in range(1, 6)]
    print("Tasks submitted...")
    
    # Get results as they complete
    for fut in as_completed(futures):
        print("->", fut.result())

Task 1 sleeping 4s
Task 2 sleeping 2s
Task 3 sleeping 1s
Tasks submitted...
Task 4 sleeping 4s
-> Task 3 finished in 1s
Task 5 sleeping 4s
-> Task 2 finished in 2s
-> Task 1 finished in 4s
-> Task 4 finished in 4s
-> Task 5 finished in 4s
